In [ ]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg")
import Tmp

In [ ]:
reload("Tmp")
include("test.jl")

In [ ]:
Lx = 14
H = Tmp.heisenberg(Lx)
eheis, vheis = eigs(H, nev=1, which=:SR)

In [ ]:
mps = Tmp.MPS(Lx, 2, vheis[:,1])
@show mps.dims
mpo = Tmp.MPO{Float64}(Lx, 2)

sz = Float64[0.5 0; 0 -0.5]
@show Tmp.measure(mps, mpo) ≈ eheis
@show Tmp.measure(mps, sz, 1)
@show Tmp.measure(mps, sz, collect(1:Lx))
correlations = Float64[]
for i=1:Lx-1
    for j=i+1:Lx
        push!(correlations, Tmp.measure(mps, [sz,sz], [i,j])[1,1])
    end
end

In [ ]:
using Plots
using LaTeXStrings
font = Plots.font("Helvetica", 12)
pyplot(guidefont=font, xtickfont=font, ytickfont=font, legendfont=font)

plot(correlations[1:Lx-1], label=L"$\langle S^z \cdot S^z \rangle$")

In [ ]:
vnes = Tmp.entropy(Tmp.entanglements!(mps))
plot(vnes)

In [ ]:
srand(0)
reload("Tmp")
# TODO:
# test canonicaize to right and left
# and center_at (how?)
Lx = 14
d = 2
mps = Tmp.MPS(Lx, d, normalize(rand(Complex128, d^Lx)))
#Tmp.display_matrices(mps,3:4)
before=Tmp.mps2ketstate(mps)

#@show 
ents = Tmp.entanglements!(mps)

#@show
Tmp.move_center!(mps, 3)

# @show ents = Tmp.entanglements!(mps)

# @show Tmp.move_center!(mps, 2)

# @show ents = Tmp.entanglements!(mps)

# @show Tmp.move_center!(mps, 1)

# @show ents = Tmp.entanglements!(mps)

# @show Tmp.move_center!(mps, 4)

# @show ents = Tmp.entanglements!(mps)

after=Tmp.mps2ketstate(mps)

@show before ≈ after
vnes = Tmp.entropy(Tmp.entanglements!(mps))
plot(vnes)

In [ ]:
# write and test the overlap function
# test and fix the apply unitary function and possibly generalize
# And move to the last step: Fishman White approach

In [ ]:
Lx=8
corr_matrix = Tmp.correlation_matrix(Lx)
display(corr_matrix)
gmps = Tmp.makeGMPS!(corr_matrix, 1.e-8)
mps = Tmp.generateMPS(gmps, 100)